In [1]:
import io
import time
import csv
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import pandas as pd
from datetime import datetime
import os
import shutil
import paramiko
import re
import math, calendar, datetime as dt
import sys
from pathlib import Path
from collections import defaultdict

c:\Users\tanzil.baraskar\AppData\Local\Programs\Python\Python310\lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\tanzil.baraskar\AppData\Local\Programs\Python\Python310\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 45.0.0.
  "class": algorithms.Blowfish,
c:\Users\tanzil.baraskar\AppData\Local\Programs\Python\Python310\lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.


In [2]:
account = 'KINESSO-KINESSO_MANAGED_REPORTING'
user = 'tanzil.baraskar@umj3.com'
authenticator = 'externalbrowser'

conn = snowflake.connector.connect(
    user=user,
    account=account,
    authenticator=authenticator,
    warehouse = 'WH_CLIENTTEAM_GILEAD_XS',
    database = 'GILEAD',
    schema = 'CUSTOM_OUTPUTS'
  
)

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://login.microsoftonline.com/d026e4c1-5892-497a-b9da-ee493c9f0364/saml2?SAMLRequest=pVPBbuIwFPyVyHtO4pi0EAuoaBFaJJZFEPbQS%2BXGL2Dh2Fnbaejf1wkgdQ%2FtZU%2B2nmc8897Y44dzJYM3MFZoNUFJhFEAqtBcqMME7fNFOEKBdUxxJrWCCXoHix6mY8sqWdNZ445qC38bsC7wFylLu4MJaoyimllhqWIVWOoKupv9WlESYcqsBeO8HLpSuBVe6%2BhcTeO4bduoHUTaHGKCMY5xFntUB%2FmBPknU32vURjtdaHmjnH1PX0gkMU47CY%2FwCpsr8VGoywi%2BU3m9gCz9meebcPN7l6NgduvuSSvbVGB2YN5EAfvt6mLAegcnocBaHV7Xl4opdgD%2BYqDWnqwOkVW6LSU7QaGruulLfheXwGOpD8LPbjmfoPokuD7zdbsapnqbnNrMPu4d44OcrZLF2S1Sty5Xz8ccyw3b6wIFf25Jky7ppbUNLFWXr%2FMlTO7CBIdkmBNMyYAmJEpG2TMK5j5foZjrmbcmeh9RJQqjrS6dVtJ307vkmNxDWiTh3SgjYZoNWfiacRYCpNmgyEo8uE%2FjLkWCLi%2BJ9kbM9H%2FnM44%2F33Z9pGuf23K%2B0VIU78FCm4q5r2NNoqSvCB6WPZRCxYSccW68Ex%2BvlLp9MsCc%2FwvONIDi6UX139

# SEARCH KEY

In [ ]:

BASE_DIR = r"C:\Users\tanzil.baraskar\OneDrive - Interpublic\Desktop\Data Feed File Downloads\SEARCH KEY"
YEAR = 2025
MONTH = 9
TABLE_A = "GILEAD.CUSTOM_OUTPUTS.DATAFEED_SEARCH_KEY"
TABLE_B = "GILEAD.CUSTOM_OUTPUTS.GILEAD_LOOKUP_SEARCH_BRAND_NAME"

# Columns in the exact order/names you want in the CSVs
DESIRED_HEADERS = [
    "Franchise",
    "Subsidiary",
    "Brand",
    "Brand Indication/Campaign",
    "Row Type",
    "Status",
    "Sync Errors",
    "Date",
    "Week Sort",
    "From",
    "To",
    "Engine",
    "Account",
    "Campaign",
    "Campaign Type",
    "Ad group",
    "Keyword",
    "Match type",
    "Media Impressions",
    "Media Clicks",
    "Click Through Rate",
    "Media Cost (USD)",
    "Cost Per Click",
    "Keyword landing page",
    "GMID",
]


def safe_token(s) -> str:
    s = "" if s is None or (isinstance(s, float) and pd.isna(s)) else str(s)
    s = s.strip().replace(" ", "_")
    s = re.sub(r"[^\w\-.]", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s or "NA"

def title_case_str(s) -> str:
    s = "" if s is None or (isinstance(s, float) and pd.isna(s)) else str(s)
    return s.strip().lower().title()

def derive_strategy(account_val, campaign_val) -> str:
    a = "" if account_val is None or (isinstance(account_val, float) and pd.isna(account_val)) else str(account_val).lower()
    c = "" if campaign_val is None or (isinstance(campaign_val, float) and pd.isna(campaign_val)) else str(campaign_val).lower()
    if "dtc" in a or "_dtc_" in c: return "DTC"
    if "dtp" in a or "_dtp_" in c: return "DTP"
    if "hcp" in a or "_hcp_" in c: return "HCP"
    return "NA"

def uniq_join(vals) -> str:
    if vals is None:
        return ""
    s = pd.Series(list(vals))
    s = s.dropna().astype(str).str.strip()
    if s.empty:
        return ""
    return "|".join(sorted(set(s)))

def write_group_csv(out_path: str, df_group: pd.DataFrame) -> int:
    """Write CSV with desired headers and return row count."""
    present_headers = [c for c in DESIRED_HEADERS if c in df_group.columns]
    g_out = df_group[present_headers].copy()
    g_out = (
        g_out.assign(__DATE_PARSED__=df_group["__DATE_PARSED__"])
             .sort_values(by="__DATE_PARSED__", na_position="last")
             .drop(columns="__DATE_PARSED__")
    )
    g_out.to_csv(out_path, index=False)
    return len(g_out)


query = f"""
SELECT
  a."Franchise",
  a."Subsidiary",
  a."Brand",
  a."Brand Indication/Campaign",
  a."Row Type",
  a."Status",
  a."Sync Errors",
  a."Date",
  a."Week Sort",
  a."From",
  a."To",
  a."Engine",
  a."Account",
  a."Campaign",
  a."Campaign Type",
  a."Ad group",
  a."Keyword",
  a."Match type",
  a."Media Impressions",
  a."Media Clicks",
  a."Click Through Rate",
  a."Media Cost (USD)",
  a."Cost Per Click",
  a."Keyword landing page",
  a.GMID,
  b.TA,
  b.BRAND_FILE_NAME
FROM {TABLE_A} a
LEFT JOIN {TABLE_B} b
  ON a."Account" = b.ACCOUNT   -- change to b."Account" if lookup column is quoted/mixed-case
WHERE EXTRACT(YEAR FROM a."Date") = {YEAR}
  AND EXTRACT(MONTH FROM a."Date") = {MONTH}
"""


def main(conn):
    df = pd.read_sql(query, conn)

    if df.empty:
        print(f"No rows for {MONTH}/{YEAR}.")
        return

    dt = pd.to_datetime(df["Date"], errors="coerce")
    if dt.isna().all():
        raise ValueError("All 'Date' values are NaT after parsing; check source data.")
    df["__DATE_PARSED__"] = dt
    df["__ACCOUNT__"]  = df["Account"]
    df["__CAMPAIGN__"] = df["Campaign"]

    df["__MONTH_YEAR__"] = df["__DATE_PARSED__"].dt.strftime("%B_%Y")
    folder_month_label = f"{df['__DATE_PARSED__'].dt.strftime('%B').iloc[0]} {int(df['__DATE_PARSED__'].dt.year.iloc[0])}"
    df["__CHANNEL__"] = "SearchKey"

    if "Campaign Type" in df.columns:
        COL_CTYPE = "Campaign Type"
    elif "Brand Type" in df.columns:
        COL_CTYPE = "Brand Type"
    else:
        COL_CTYPE = "__CTYPE__"
        df[COL_CTYPE] = "NA"

    df["__STRATEGY__"] = df.apply(lambda r: derive_strategy(r["__ACCOUNT__"], r["__CAMPAIGN__"]), axis=1)

    df["TA"] = df["TA"].fillna("") if "TA" in df.columns else ""
    brand_file_col = "BRAND_FILE_NAME" if "BRAND_FILE_NAME" in df.columns else None
    has_brand_file = brand_file_col is not None

    # Output folder
    out_base = os.path.join(BASE_DIR, f"{folder_month_label} Data Feed Delivery", "SearchKey")
    os.makedirs(out_base, exist_ok=True)

    summary = {}

    def add_to_summary(fname, fpath, subset):
        entry = summary.setdefault(fname, {
            "Full Path": fpath,
            "Rows": 0,
            "Brand": set(),
            "BIC": set(),
            "CType": set(),
            "Account": set(),
            "TA": set(),
            "Strategy": set(),
            "Channel": set(),
            "Month_Year": set(),
            "LookupBrandFile": set(),
        })
        entry["Rows"] += len(subset)
        for col, key in [
            ("Brand", "Brand"),
            ("Brand Indication/Campaign", "BIC"),
            (COL_CTYPE, "CType"),
            ("Account", "Account"),
            ("TA", "TA"),
            ("__STRATEGY__", "Strategy"),
            ("__CHANNEL__", "Channel"),
            ("__MONTH_YEAR__", "Month_Year"),
        ]:
            if col in subset.columns:
                entry[key].update(subset[col].dropna().astype(str).str.strip().tolist())
        if has_brand_file and brand_file_col in subset.columns:
            entry["LookupBrandFile"].update(subset[brand_file_col].dropna().astype(str).str.strip().tolist())

    files_written = 0
    rows_written = 0

    # -------- Pass 1: BRANDED (group WITHOUT brand_file) --------
    branded_mask = df[COL_CTYPE].astype(str).str.upper() == "BRANDED"
    df_b = df[branded_mask].copy()
    if not df_b.empty:
        group_cols_branded = ["TA", "__STRATEGY__", "__CHANNEL__", COL_CTYPE, "__MONTH_YEAR__"]
        for keys, g in df_b.groupby(group_cols_branded, dropna=False):
            ta, strat, channel, ctype_val, month_year = keys
            ctype_tc = title_case_str(ctype_val)
            parts = [safe_token(ta), safe_token(strat), safe_token(channel), ctype_tc or "NA", safe_token(month_year)]
            filename = "_".join([p for p in parts if p]) + ".csv"
            out_path = os.path.join(out_base, filename)

            n = write_group_csv(out_path, g)
            files_written += 1
            rows_written += n
            print(f"Wrote (BRANDED): {out_path}  (rows: {n})")

            add_to_summary(filename, out_path, g)

    # -------- Pass 2: UNBRANDED (group WITH brand_file) --------
    unbranded_mask = df[COL_CTYPE].astype(str).str.upper() == "UNBRANDED"
    df_u = df[unbranded_mask].copy()
    if not df_u.empty:
        if not has_brand_file:
            # if lookup absent, still create a placeholder col to keep behavior stable
            df_u[brand_file_col] = ""
        group_cols_unbranded = ["TA", "__STRATEGY__", "__CHANNEL__", COL_CTYPE, "__MONTH_YEAR__", brand_file_col]
        for keys, g in df_u.groupby(group_cols_unbranded, dropna=False):
            ta, strat, channel, ctype_val, month_year, brand_file = keys
            ctype_tc = title_case_str(ctype_val)
            parts = [
                safe_token(ta),
                safe_token(strat),
                safe_token(channel),
                ctype_tc or "NA",
                safe_token(brand_file),
                safe_token(month_year),
            ]
            filename = "_".join([p for p in parts if p]) + ".csv"
            out_path = os.path.join(out_base, filename)

            n = write_group_csv(out_path, g)
            files_written += 1
            rows_written += n
            print(f"Wrote (UNBRANDED): {out_path}  (rows: {n})")

            add_to_summary(filename, out_path, g)

    # -------- Pass 3: OTHER ctypes (group WITHOUT brand_file) --------
    others_mask = ~(branded_mask | unbranded_mask)
    df_o = df[others_mask].copy()
    if not df_o.empty:
        group_cols_other = ["TA", "__STRATEGY__", "__CHANNEL__", COL_CTYPE, "__MONTH_YEAR__"]
        for keys, g in df_o.groupby(group_cols_other, dropna=False):
            ta, strat, channel, ctype_val, month_year = keys
            ctype_tc = title_case_str(ctype_val)
            parts = [safe_token(ta), safe_token(strat), safe_token(channel), ctype_tc or "NA", safe_token(month_year)]
            filename = "_".join([p for p in parts if p]) + ".csv"
            out_path = os.path.join(out_base, filename)

            n = write_group_csv(out_path, g)
            files_written += 1
            rows_written += n
            print(f"Wrote (OTHER): {out_path}  (rows: {n})")

            add_to_summary(filename, out_path, g)

    print(f"\nDone. Files: {files_written}  Total rows written: {rows_written}")

    # -------- Write aggregated summary (one row per file) --------
    if summary:
        rows = []
        for fname, d in summary.items():
            rows.append({
                "File Name": fname,
                "Full Path": d["Full Path"],
                "Rows": d["Rows"],
                "Brand (unique)": uniq_join(d["Brand"]),
                "Brand Indication/Campaign (unique)": uniq_join(d["BIC"]),
                "Campaign Type (unique)": uniq_join(d["CType"]),
                "Account (unique)": uniq_join(d["Account"]),
                "TA": uniq_join(d["TA"]),
                "Strategy": uniq_join(d["Strategy"]),
                "Channel": uniq_join(d["Channel"]),
                "Month_Year": uniq_join(d["Month_Year"]),
                "Lookup Brand File Name": uniq_join(d["LookupBrandFile"]),
            })
        summary_df = pd.DataFrame(rows)
        summary_name = f"SearchKey_File_Summary_{df['__DATE_PARSED__'].dt.strftime('%B_%Y').iloc[0]}.csv"
        summary_path = os.path.join(out_base, summary_name)
        summary_df.to_csv(summary_path, index=False)
        print(f"Summary written: {summary_path}")
    else:
        print("No files written; skipping summary.")


if __name__ == "__main__":
    try:
        conn  # noqa: F821
    except NameError:
        raise RuntimeError("Please create a Snowflake connection named `conn` before running this script.")
    main(conn)


# SEARCH GEO

In [ ]:
BASE_DIR = r"C:\Users\tanzil.baraskar\OneDrive - Interpublic\Desktop\Data Feed File Downloads\SEARCH GEO"
YEAR = 2025
MONTH = 8
TABLE_A = "GILEAD.CUSTOM_OUTPUTS.DATAFEED_SEARCH_GEO"
TABLE_B = "GILEAD.CUSTOM_OUTPUTS.GILEAD_LOOKUP_SEARCH_BRAND_NAME"

# The exact headers (and order) you want in the CSVs (GEO schema)
DESIRED_HEADERS = [
    "Franchise",
    "Subsidiary",
    "Brand",
    "Brand Indication/Campaign",
    "Row Type",
    "Status",
    "Sync Errors",
    "Date",
    "Week Sort",
    "From",
    "To",
    "Engine",
    "Account",
    "Campaign",
    "Campaign Type",
    "Country",
    "Country ID",
    "DMA",
    "DMA Code",
    "State / Region",
    "State / Region ID",
    "City",
    "City ID",
    "Media Impressions",
    "Media Clicks",
    "Click Through Rate",
    "Media Cost (USD)",
    "Cost Per Click"

]


def safe_token(s) -> str:
    s = "" if s is None or (isinstance(s, float) and pd.isna(s)) else str(s)
    s = s.strip().replace(" ", "_")
    s = re.sub(r"[^\w\-.]", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s or "NA"

def title_case_str(s) -> str:
    s = "" if s is None or (isinstance(s, float) and pd.isna(s)) else str(s)
    return s.strip().lower().title()

def derive_strategy(account_val, campaign_val) -> str:
    a = "" if account_val is None or (isinstance(account_val, float) and pd.isna(account_val)) else str(account_val).lower()
    c = "" if campaign_val is None or (isinstance(campaign_val, float) and pd.isna(campaign_val)) else str(campaign_val).lower()
    if "dtc" in a or "_dtc_" in c: return "DTC"
    if "dtp" in a or "_dtp_" in c: return "DTP"
    if "hcp" in a or "_hcp_" in c: return "HCP"
    return "NA"

def uniq_join(vals) -> str:
    if vals is None:
        return ""
    s = pd.Series(list(vals))
    s = s.dropna().astype(str).str.strip()
    if s.empty:
        return ""
    # match Search Key: dedup + SORTED
    return "|".join(sorted(set(s)))

def write_group_csv(out_path: str, df_group: pd.DataFrame) -> int:
    """Write CSV with desired headers and return row count."""
    present_headers = [c for c in DESIRED_HEADERS if c in df_group.columns]
    g_out = df_group[present_headers].copy()
    g_out = (
        g_out.assign(__DATE_PARSED__=df_group["__DATE_PARSED__"])
             .sort_values(by="__DATE_PARSED__", na_position="last")
             .drop(columns="__DATE_PARSED__")
    )
    g_out.to_csv(out_path, index=False)
    return len(g_out)


query = f"""
SELECT
  a."Franchise",
  a."Subsidiary",
  a."Brand",
  a."Brand Indication/Campaign",
  a."Row Type",
  a."Status",
  a."Sync Errors",
  a."Date",
  a."Week Sort",
  a."From",
  a."To",
  a."Engine",
  a."Account",
  a."Campaign",
  a."Campaign Type",
  a."Country",
  a."Country ID",
  a.DMA,
  a."DMA Code",
  a."State / Region",
  a."State / Region ID",
  a."City",
  a."City ID",
  a."Media Impressions",
  a."Media Clicks",
  a."Click Through Rate",
  a."Media Cost (USD)",
  a."Cost Per Click",
  b.TA,
  b.BRAND_FILE_NAME
FROM {TABLE_A} a
LEFT JOIN {TABLE_B} b
  ON a."Account" = b.ACCOUNT   -- change to b."Account" if lookup column is quoted/mixed-case
WHERE EXTRACT(YEAR FROM a."Date") = {YEAR}
  AND EXTRACT(MONTH FROM a."Date") = {MONTH}
"""

def main(conn):
    df = pd.read_sql(query, conn)

    if df.empty:
        print(f"No rows for {MONTH}/{YEAR}.")
        return

    # ---- INTERNAL HELPER COLUMNS ----
    dt = pd.to_datetime(df["Date"], errors="coerce")
    if dt.isna().all():
        raise ValueError("All 'Date' values are NaT after parsing; check source data.")
    df["__DATE_PARSED__"] = dt
    df["__ACCOUNT__"]  = df["Account"]
    df["__CAMPAIGN__"] = df["Campaign"]

    df["__MONTH_YEAR__"] = df["__DATE_PARSED__"].dt.strftime("%B_%Y")
    folder_month_label = f"{df['__DATE_PARSED__'].dt.strftime('%B').iloc[0]} {int(df['__DATE_PARSED__'].dt.year.iloc[0])}"
    df["__CHANNEL__"] = "SearchGeo"

    # Campaign type column selection
    if "Campaign Type" in df.columns:
        COL_CTYPE = "Campaign Type"
    elif "Brand Type" in df.columns:
        COL_CTYPE = "Brand Type"
    else:
        COL_CTYPE = "__CTYPE__"
        df[COL_CTYPE] = "NA"

    # Strategy
    df["__STRATEGY__"] = df.apply(lambda r: derive_strategy(r["__ACCOUNT__"], r["__CAMPAIGN__"]), axis=1)

    # Lookup columns
    df["TA"] = df["TA"].fillna("") if "TA" in df.columns else ""
    brand_file_col = "BRAND_FILE_NAME" if "BRAND_FILE_NAME" in df.columns else None
    has_brand_file = brand_file_col is not None

    # Output folder
    out_base = os.path.join(BASE_DIR, f"{folder_month_label} Data Feed Delivery", "SearchGeo")
    os.makedirs(out_base, exist_ok=True)

    # -------- Summary aggregator keyed by filename (same as Search Key) --------
    summary = {}

    def add_to_summary(fname, fpath, subset):
        entry = summary.setdefault(fname, {
            "Full Path": fpath,
            "Rows": 0,
            "Brand": set(),
            "BIC": set(),
            "CType": set(),
            "Account": set(),
            "TA": set(),
            "Strategy": set(),
            "Channel": set(),
            "Month_Year": set(),
            "LookupBrandFile": set(),
        })
        entry["Rows"] += len(subset)
        for col, key in [
            ("Brand", "Brand"),
            ("Brand Indication/Campaign", "BIC"),
            (COL_CTYPE, "CType"),
            ("Account", "Account"),
            ("TA", "TA"),
            ("__STRATEGY__", "Strategy"),
            ("__CHANNEL__", "Channel"),
            ("__MONTH_YEAR__", "Month_Year"),
        ]:
            if col in subset.columns:
                entry[key].update(subset[col].dropna().astype(str).str.strip().tolist())
        if has_brand_file and brand_file_col in subset.columns:
            entry["LookupBrandFile"].update(subset[brand_file_col].dropna().astype(str).str.strip().tolist())

    files_written = 0
    rows_written = 0

    # -------- Pass 1: BRANDED (group WITHOUT brand_file) --------
    branded_mask = df[COL_CTYPE].astype(str).str.upper() == "BRANDED"
    df_b = df[branded_mask].copy()
    if not df_b.empty:
        group_cols_branded = ["TA", "__STRATEGY__", "__CHANNEL__", COL_CTYPE, "__MONTH_YEAR__"]
        for keys, g in df_b.groupby(group_cols_branded, dropna=False):
            ta, strat, channel, ctype_val, month_year = keys
            ctype_tc = title_case_str(ctype_val)
            parts = [safe_token(ta), safe_token(strat), safe_token(channel), ctype_tc or "NA", safe_token(month_year)]
            filename = "_".join([p for p in parts if p]) + ".csv"
            out_path = os.path.join(out_base, filename)

            n = write_group_csv(out_path, g)
            files_written += 1
            rows_written += n
            print(f"Wrote (BRANDED): {out_path}  (rows: {n})")

            add_to_summary(filename, out_path, g)

    # -------- Pass 2: UNBRANDED (group WITH brand_file) --------
    unbranded_mask = df[COL_CTYPE].astype(str).str.upper() == "UNBRANDED"
    df_u = df[unbranded_mask].copy()
    if not df_u.empty:
        if not has_brand_file:
            # if lookup absent, still create a placeholder col to keep behavior stable
            df_u[brand_file_col] = ""
        group_cols_unbranded = ["TA", "__STRATEGY__", "__CHANNEL__", COL_CTYPE, "__MONTH_YEAR__", brand_file_col]
        for keys, g in df_u.groupby(group_cols_unbranded, dropna=False):
            ta, strat, channel, ctype_val, month_year, brand_file = keys
            ctype_tc = title_case_str(ctype_val)
            parts = [
                safe_token(ta),
                safe_token(strat),
                safe_token(channel),
                ctype_tc or "NA",
                safe_token(brand_file),
                safe_token(month_year),
            ]
            filename = "_".join([p for p in parts if p]) + ".csv"
            out_path = os.path.join(out_base, filename)

            n = write_group_csv(out_path, g)
            files_written += 1
            rows_written += n
            print(f"Wrote (UNBRANDED): {out_path}  (rows: {n})")

            add_to_summary(filename, out_path, g)

    # -------- Pass 3: OTHER ctypes (group WITHOUT brand_file) --------
    others_mask = ~(branded_mask | unbranded_mask)
    df_o = df[others_mask].copy()
    if not df_o.empty:
        group_cols_other = ["TA", "__STRATEGY__", "__CHANNEL__", COL_CTYPE, "__MONTH_YEAR__"]
        for keys, g in df_o.groupby(group_cols_other, dropna=False):
            ta, strat, channel, ctype_val, month_year = keys
            ctype_tc = title_case_str(ctype_val)
            parts = [safe_token(ta), safe_token(strat), safe_token(channel), ctype_tc or "NA", safe_token(month_year)]
            filename = "_".join([p for p in parts if p]) + ".csv"
            out_path = os.path.join(out_base, filename)

            n = write_group_csv(out_path, g)
            files_written += 1
            rows_written += n
            print(f"Wrote (OTHER): {out_path}  (rows: {n})")

            add_to_summary(filename, out_path, g)

    print(f"\nDone. Files: {files_written}  Total rows written: {rows_written}")

    # -------- Write aggregated summary (one row per file) --------
    if summary:
        rows = []
        for fname, d in summary.items():
            rows.append({
                "File Name": fname,
                "Full Path": d["Full Path"],
                "Rows": d["Rows"],
                "Brand (unique)": uniq_join(d["Brand"]),
                "Brand Indication/Campaign (unique)": uniq_join(d["BIC"]),
                "Campaign Type (unique)": uniq_join(d["CType"]),
                "Account (unique)": uniq_join(d["Account"]),
                "TA": uniq_join(d["TA"]),
                "Strategy": uniq_join(d["Strategy"]),
                "Channel": uniq_join(d["Channel"]),
                "Month_Year": uniq_join(d["Month_Year"]),
                "Lookup Brand File Name": uniq_join(d["LookupBrandFile"]),
            })
        summary_df = pd.DataFrame(rows)
        summary_name = f"SearchGeo_File_Summary_{df['__DATE_PARSED__'].dt.strftime('%B_%Y').iloc[0]}.csv"
        summary_path = os.path.join(out_base, summary_name)
        summary_df.to_csv(summary_path, index=False)
        print(f"Summary written: {summary_path}")
    else:
        print("No files written; skipping summary.")


if __name__ == "__main__":
    try:
        conn  # noqa: F821
    except NameError:
        raise RuntimeError("Please create a Snowflake connection named `conn` before running this script.")
    main(conn)


# DCM NON-DMA AUDIO

In [ ]:
BASE_DIR = r"C:\Users\tanzil.baraskar\OneDrive - Interpublic\Desktop\Data Feed File Downloads\Non-DMA AUD"
YEAR = 2025
MONTH = 8
TABLE_A = "GILEAD.CUSTOM_OUTPUTS.DATAFEED_DCM_AUDIO"
CHANNEL_FILTER = "Audio"

def safe_token(s) -> str:
    s = "" if s is None or (isinstance(s, float) and pd.isna(s)) else str(s)
    s = s.strip().replace(" ", "_")
    s = re.sub(r"[^\w\-.]", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s or "NA"

def title_case_str(s) -> str:
    s = "" if s is None or (isinstance(s, float) and pd.isna(s)) else str(s)
    return s.strip().lower().title()

def resolve_col(df, candidates, allow_fuzzy=True):
    """
    Return the actual column name from df that matches any candidate (case/space-insensitive).
    """
    cols = list(df.columns)
    norm = {c: re.sub(r'\s+', '', c).lower() for c in cols}

    # exact (case-insensitive)
    for cand in candidates:
        for c in cols:
            if c.lower() == cand.lower():
                return c

    if allow_fuzzy:
        cand_norms = [re.sub(r'\s+', '', c).lower() for c in candidates]
        for c, n in norm.items():
            if n in cand_norms:
                return c
        # partial match (e.g., 'Franchise ' with trailing space)
        for cand in cand_norms:
            for c, n in norm.items():
                if n == cand or n.startswith(cand) or cand.startswith(n):
                    return c
    return None

# Define desired output schema (exact headers) with candidate source names and type
OUTPUT_SCHEMA = [
    # (Exact Output Header, ["candidate names ..."], type: "text"|"num"|"date")
    ("Franchise", ["FRANCHISE","Franchise","FRANCHISE_NAME","Brand_Franchise","BRAND_FRANCHISE","BRAND_FRANCAHISE","BRAND"], "text"),
    ("Subsidiary", ["SUBSIDIARY","Subsidiary","SUBSIDIARY_NAME"], "text"),
    ("Brand", ["BRAND","Brand"], "text"),
    ("Brand Indication/Campaign", ["Brand Indication/Campaign","Brand Indication","BRAND_INDICATION","BRAND_INDICATION_CAMPAIGN","CAMPAIGN_NAME","Campaign Name"], "text"),
    ("Date", ["DATE","Date","DAY","Day"], "date"),
    ("Campaign Type", ["Campaign Type ","Campaign Type","CAMPAIGN_TYPE","BRAND_TYPE","Brand Type"], "text"),
    ("Campaign", ["CAMPAIGN","Campaign","Campaign Name","CAMPAIGN_NAME"], "text"),
    ("Campaign ID", ["CAMPAIGN_ID","Campaign ID","CampaignId"], "text"),
    ("Site / Publisher ID", ["Site/Publisher ID","PUBLISHER_ID","Publisher Id","Site Id"], "text"),
    ("Site / Publisher", ["Site/Publisher","SITE/PUBLISHER","PUBLISHER","Publisher","Site"], "text"),
    ("DMA", ["DMA","Dma","DMA_REGION","DMA Name"], "text"),
    ("DMA Code", ["DMA Code","DMA_CODE","DMACode","DMA_ID","DMA Id"], "text"),
    ("Package/Roadblock", ["Package/Roadblock","PACKAGE/ROADBLOCK","PACKAGE_ROADBLOCK","PACKAGE_GROUP_NAME","PACKAGE_NAME"], "text"),
    ("Package/Roadblock ID", ["Package/Roadblock ID","PACKAGE_ROADBLOCK_ID","PACKAGE_ID","Package Id"], "text"),
    ("Placement", ["PLACEMENT","Placement","PLACEMENT_NAME"], "text"),
    ("Placement ID", ["PLACEMENT_ID","Placement ID","PlacementId"], "text"),
    ("Target Tactic", ["Target Tactic","TARGET_TACTIC","Tactic","TACTIC"], "text"),
    ("Creative", ["CREATIVE","Creative","CREATIVE_NAME"], "text"),
    ("Creative ID", ["CREATIVE_ID","Creative ID","CreativeId"], "text"),
    ("Ad", ["Ad","AD","AD_NAME","Ad Name"], "text"),
    ("Media Impressions", ["Media Impressions","MEDIA_IMPRESSIONS","IMPRESSIONS","Impressions"], "num"),
    ("Media Clicks", ["Media Clicks","MEDIA_CLICKS","CLICKS","Clicks"], "num"),
    ("Audio Plays", ["Audio Plays","AUDIO_PLAYS","PLAYS","AudioPlays"], "num"),
    ("Audio Completes", ["Audio Completes","AUDIO_COMPLETES","COMPLETES","AudioCompletes"], "num"),
    ("GMID", ["GMID","Gmid"], "text"),
]

query = f"""
SELECT *
FROM {TABLE_A} a
WHERE EXTRACT(YEAR FROM a.DATE) = {YEAR}
  AND EXTRACT(MONTH FROM a.DATE) = {MONTH}
  AND a."Channel" = '{CHANNEL_FILTER}'
"""
df = pd.read_sql(query, conn)

if df.empty:
    print(f"No rows for {MONTH}/{YEAR} with Channel = '{CHANNEL_FILTER}'.")
else:
    # Detect Franchise / Strategy / Campaign Type used for file naming
    franchise_col = resolve_col(df, ["FRANCHISE","Franchise","FRANCHISE_NAME","Brand_Franchise","BRAND_FRANCHISE","BRAND"])
    if franchise_col is None:
        df["__FRANCHISE__"] = "NA"
        franchise_col = "__FRANCHISE__"

    strat_col = resolve_col(df, ["HCP/DTP","HCP_DTP","Strategy"]) or "__STRATEGY__"
    if strat_col == "__STRATEGY__": df["__STRATEGY__"] = "NA"

    ctype_col = resolve_col(df, ["Campaign Type ","Campaign Type","CAMPAIGN_TYPE","BRAND_TYPE"]) or "__CTYPE__"
    if ctype_col == "__CTYPE__": df["__CTYPE__"] = "NA"

    # Month/year helpers
    date_src = resolve_col(df, ["DATE","Date","DAY","Day"])
    if date_src is None:
        raise ValueError("DATE column not found.")
    dt = pd.to_datetime(df[date_src], errors="coerce")
    if dt.isna().all():
        raise ValueError("DATE column could not be parsed as dates.")

    df["__MONTH_NAME__"] = dt.dt.strftime("%B")
    folder_month_label = f"{dt.dt.strftime('%B').iloc[0]} {dt.dt.year.iloc[0]}"

    # Output folder
    out_base = os.path.join(BASE_DIR, f"{folder_month_label} Data Feed Delivery", "Non-DMA AUD")
    os.makedirs(out_base, exist_ok=True)

    # Build a mapping {output_header: source_col_or_None}
    source_map = {}
    for out_name, candidates, _dtype in OUTPUT_SCHEMA:
        found = resolve_col(df, candidates)
        source_map[out_name] = found  # can be None

    # Quick sanity prints
    print(f"Franchise column detected as: {franchise_col}")
    print(f"Non-null franchise rows: {df[franchise_col].notna().sum()} / {len(df)}")
    print("Source column mapping (None means we will fill defaults):")
    for k, v in source_map.items():
        print(f"  {k}  <-  {v}")

    # Grouping: Franchise, HCP/DTP, Campaign Type, Month
    group_cols = [franchise_col, strat_col, ctype_col, "__MONTH_NAME__"]

    files_written = 0
    rows_written = 0

    for keys, g in df.groupby(group_cols, dropna=False):
        franchise, strat, ctype_val, month_name = keys

        ctype_tc = title_case_str(ctype_val)
        strat_up = (str(strat).strip().upper() or "NA")

        # Prepare output frame with exact columns in required order
        out_frames = {}
        for out_name, _cands, dtype in OUTPUT_SCHEMA:
            src = source_map[out_name]
            if src is not None and src in g.columns:
                col = g[src].copy()
            else:
                # create default column
                if dtype == "num":
                    col = pd.Series(0, index=g.index, dtype="Int64")
                elif dtype == "date":
                    # coerce from current group date source if present
                    if date_src in g.columns:
                        col = pd.to_datetime(g[date_src], errors="coerce")
                    else:
                        col = pd.NaT
                else:
                    col = pd.Series("NA", index=g.index, dtype="object")
            out_frames[out_name] = col

        g_out = pd.DataFrame(out_frames)

        # Ensure DATE is date-like and not object
        if not pd.api.types.is_datetime64_any_dtype(g_out["Date"]):
            g_out["Date"] = pd.to_datetime(g_out["Date"], errors="coerce")

        # Final sort + write
        g_out = g_out.sort_values(by=["Date"], na_position="last").reset_index(drop=True)

        # Franchise_HCP-or-DTP_AUD_Branded-or-Unbranded_Month_2025_Non-DMA.csv
        filename = "_".join([
            safe_token(franchise),
            safe_token(strat_up),
            "AUD",
            safe_token(ctype_tc if ctype_tc else "NA"),
            safe_token(month_name),
            str(YEAR),
            "Non-DMA"
        ]) + ".csv"

        out_path = os.path.join(out_base, filename)
        g_out.to_csv(out_path, index=False)

        files_written += 1
        rows_written += len(g_out)
        print(f"Wrote: {out_path}  (rows: {len(g_out)})")

    print(f"\nDone. Files: {files_written}  Total rows written: {rows_written}")


# DCM

In [ ]:
BASE_DIR = r"C:\Users\tanzil.baraskar\OneDrive - Interpublic\Desktop\Data Feed File Downloads\DCM"
TABLE_A = "DATAFEED_DCM"
LOOKUP_TABLE = "GILEAD_LOOKUP_DCM_BRAND_NAME"   

YEAR = 2025
MONTH = 8
SELECTED_CHANNELS = ["Audio", "CTV", "Custom", "Display", "FEP", "Newsletter", "OLV"]

CHUNKSIZE = 250_000
EXCEL_ROW_CAP = 1_048_574

# Channels where we should NOT include the "Video Completes" column in the exported CSV
NONVIDEO_CHANNELS = {"display", "newsletter","custom"}

# Channels that should use Publisher BEFORE Publisher ID in the schema
CHANNELS_PUBLISHER_FIRST = {"custom", "display", "newsletter"}
# ----------------------------------------

def safe_token(s) -> str:
    s = "" if s is None or (isinstance(s, float) and pd.isna(s)) else str(s)
    s = s.strip().replace(" ", "_")
    s = re.sub(r"[^\w\-.]", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s or "NA"

def title_case_str(s) -> str:
    s = "" if s is None or (isinstance(s, float) and pd.isna(s)) else str(s)
    return s.strip().lower().title()

def resolve_col(df, candidates, allow_fuzzy=True):
    cols = list(df.columns)
    norm = {c: re.sub(r"\s+", "", c).lower() for c in cols}
    # exact (case-insensitive)
    for cand in candidates:
        for c in cols:
            if c.lower() == cand.lower():
                return c
    if allow_fuzzy:
        cand_norms = [re.sub(r"\s+", "", c).lower() for c in candidates]
        # whitespace-insensitive
        for c, n in norm.items():
            if n in cand_norms:
                return c
        # prefix-ish matches
        for cand in cand_norms:
            for c, n in norm.items():
                if n == cand or n.startswith(cand) or cand.startswith(n):
                    return c
    return None

def num_files_from_count_alteryx(count: int) -> int:
    ratio = count / EXCEL_ROW_CAP
    if ratio <= 1: return 1
    elif ratio <= 2: return 3
    else: return min(10, int(math.ceil(ratio)) + 1)

def month_day_ranges(year: int, month: int, n: int):
    last = calendar.monthrange(year, month)[1]
    if n == 1: return [(1, last)]
    if n == 2: return [(1, 15), (16, last)]
    if n == 3: return [(1, 10), (11, 20), (21, last)]
    if n == 4: return [(1, 7), (8, 14), (15, 21), (22, last)]
    if n == 5: return [(1, 6), (7, 12), (13, 18), (19, 24), (25, last)]
    # balanced split fallback
    edges = [1]
    for k in range(1, n):
        d = math.floor(1 + (last - 1) * (k / n))
        if d <= edges[-1]: d = edges[-1] + 1
        edges.append(min(d, last))
    edges.append(last + 1)
    ranges = []
    for i in range(n):
        start = edges[i]
        end = min(edges[i+1] - 1, last)
        if end < start: end = start
        ranges.append((start, end))
    ranges[-1] = (ranges[-1][0], last)
    return ranges

def day_of_month(series):
    s = pd.to_datetime(series, errors="coerce")
    return s.dt.day

# --------- Output Schemas ----------
OUTPUT_SCHEMA_STANDARD = [
    ("Franchise", ["FRANCHISE","Franchise","FRANCHISE_NAME","Brand_Franchise","BRAND_FRANCHISE","BRAND_FRANCAHISE","BRAND"], "text"),
    ("Subsidiary", ["SUBSIDIARY","Subsidiary","SUBSIDIARY_NAME"], "text"),
    ("Brand", ["BRAND","Brand"], "text"),
    ("Brand Indication/Campaign", ["Brand Indication/Campaign","Brand Indication","BRAND_INDICATION","BRAND_INDICATION_CAMPAIGN","CAMPAIGN_NAME","Campaign Name"], "text"),
    ("Date", ["DATE","Date","DAY","Day"], "date"),
    ("Campaign Type", ["Campaign Type ","Campaign Type","CAMPAIGN_TYPE","BRAND_TYPE","Brand Type"], "text"),
    ("Campaign", ["CAMPAIGN","Campaign","Campaign Name","CAMPAIGN_NAME"], "text"),
    ("Campaign ID", ["CAMPAIGN_ID","Campaign ID","CampaignId"], "text"),
    ("Site / Publisher ID", ["Site/Publisher ID","SITE/PUBLISHER ID","PUBLISHER_ID","Publisher Id","Site Id","SITE_ID"], "text"),
    ("Site / Publisher", ["Site/Publisher","SITE/PUBLISHER","PUBLISHER","Publisher","Site"], "text"),
    ("DMA", ["DMA","Dma","DMA_REGION","DMA Name"], "text"),
    ("DMA Code", ["DMA Code","DMA_CODE","DMACode","DMA_ID","DMA Id"], "text"),
    ("Package/Roadblock", ["Package/Roadblock","PACKAGE/ROADBLOCK","PACKAGE_ROADBLOCK","PACKAGE_GROUP_NAME","PACKAGE_NAME"], "text"),
    ("Package/Roadblock ID", ["Package/Roadblock ID","PACKAGE_ROADBLOCK_ID","PACKAGE_ID","Package Id"], "text"),
    ("Placement", ["PLACEMENT","Placement","PLACEMENT_NAME"], "text"),
    ("Placement ID", ["PLACEMENT_ID","Placement ID","PlacementId"], "text"),
    ("Target Tactic", ["Target Tactic","TARGET_TACTIC","Tactic","TACTIC"], "text"),
    ("Creative", ["CREATIVE","Creative","CREATIVE_NAME"], "text"),
    ("Creative ID", ["CREATIVE_ID","Creative ID","CreativeId"], "text"),
    ("Ad", ["Ad","AD","AD_NAME","Ad Name"], "text"),
    ("Media Impressions", ["Media Impressions","MEDIA_IMPRESSIONS","IMPRESSIONS","Impressions"], "num"),
    ("Media Clicks", ["Media Clicks","MEDIA_CLICKS","CLICKS","Clicks"], "num"),
    ("Video Views", ["Video Views","VIDEO_VIEWS","VIDEO VIEWS","Views","VIEWS","VIDEO_STARTS"], "num"),
    ("Video Completes", ["Video Completes","VIDEO_COMPLETES","VIDEO COMPLETES","Completions","COMPLETIONS","VIDEO_COMPLETIONS"], "num"),
    ("GMID", ["GMID","Gmid"], "text"),
]

# Publisher-first schema (Name before ID) for Custom/Display/Newsletter
OUTPUT_SCHEMA_PUBLISHER_FIRST = [
    ("Franchise", ["FRANCHISE","Franchise","FRANCHISE_NAME","Brand_Franchise","BRAND_FRANCHISE","BRAND_FRANCAHISE","BRAND"], "text"),
    ("Subsidiary", ["SUBSIDIARY","Subsidiary","SUBSIDIARY_NAME"], "text"),
    ("Brand", ["BRAND","Brand"], "text"),
    ("Brand Indication/Campaign", ["Brand Indication/Campaign","Brand Indication","BRAND_INDICATION","BRAND_INDICATION_CAMPAIGN","CAMPAIGN_NAME","Campaign Name"], "text"),
    ("Date", ["DATE","Date","DAY","Day"], "date"),
    ("Campaign Type", ["Campaign Type ","Campaign Type","CAMPAIGN_TYPE","BRAND_TYPE","Brand Type"], "text"),
    ("Campaign", ["CAMPAIGN","Campaign","Campaign Name","CAMPAIGN_NAME"], "text"),
    ("Campaign ID", ["CAMPAIGN_ID","Campaign ID","CampaignId"], "text"),
    ("Site / Publisher", ["Site/Publisher","SITE/PUBLISHER","PUBLISHER","Publisher","Site"], "text"),
    ("Site / Publisher ID", ["Site/Publisher ID","SITE/PUBLISHER ID","PUBLISHER_ID","Publisher Id","Site Id","SITE_ID"], "text"),
    ("DMA", ["DMA","Dma","DMA_REGION","DMA Name"], "text"),
    ("DMA Code", ["DMA Code","DMA_CODE","DMACode","DMA_ID","DMA Id"], "text"),
    ("Package/Roadblock", ["Package/Roadblock","PACKAGE/ROADBLOCK","PACKAGE_ROADBLOCK","PACKAGE_GROUP_NAME","PACKAGE_NAME"], "text"),
    ("Package/Roadblock ID", ["Package/Roadblock ID","PACKAGE_ROADBLOCK_ID","PACKAGE_ID","Package Id"], "text"),
    ("Placement", ["PLACEMENT","Placement","PLACEMENT_NAME"], "text"),
    ("Placement ID", ["PLACEMENT_ID","Placement ID","PlacementId"], "text"),
    ("Target Tactic", ["Target Tactic","TARGET_TACTIC","Tactic","TACTIC"], "text"),
    ("Creative", ["CREATIVE","Creative","CREATIVE_NAME"], "text"),
    ("Creative ID", ["CREATIVE_ID","Creative ID","CreativeId"], "text"),
    ("Ad", ["Ad","AD","AD_NAME","Ad Name"], "text"),
    ("Media Impressions", ["Media Impressions","MEDIA_IMPRESSIONS","IMPRESSIONS","Impressions"], "num"),
    ("Media Clicks", ["Media Clicks","MEDIA_CLICKS","CLICKS","Clicks"], "num"),
    ("Video Views", ["Video Views","VIDEO_VIEWS","VIDEO VIEWS","Views","VIEWS","VIDEO_STARTS"], "num"),
    ("Video Completes", ["Video Completes","VIDEO_COMPLETES","VIDEO COMPLETES","Completions","COMPLETIONS","VIDEO_COMPLETIONS"], "num"),
    ("GMID", ["GMID","Gmid"], "text"),
]

month_start = dt.date(YEAR, MONTH, 1)
next_month_start = dt.date(YEAR + (MONTH == 12), (MONTH % 12) + 1, 1)

chan_list = [c.replace("'", "''") for c in SELECTED_CHANNELS]
chan_in = ", ".join(f"'{c}'" for c in chan_list)

query = f"""
SELECT *
FROM {TABLE_A} a
WHERE a.DATE >= TO_DATE('{month_start:%Y-%m-%d}')
  AND a.DATE <  TO_DATE('{next_month_start:%Y-%m-%d}')
  AND a."Channel" IN ({chan_in})
"""

# ----------------- Load lookup once (Campaign → Brand File Name, TA) -----------------
lookup_df = pd.read_sql(f"SELECT * FROM {LOOKUP_TABLE}", conn)

lkp_campaign_col = resolve_col(lookup_df, ["CAMPAIGN_NAME","Campaign Name","Campaign","BRAND_INDICATION_CAMPAIGN","Brand Indication/Campaign"])
if lkp_campaign_col is None:
    raise ValueError(f"Could not find campaign column in {LOOKUP_TABLE}")

lkp_brandfile_col = resolve_col(lookup_df, ["Brand File Name","BRAND_FILE_NAME","Brand_File_Name","BrandFileName"])
if lkp_brandfile_col is None:
    raise ValueError(f"Could not find 'Brand File Name' column in {LOOKUP_TABLE}")

lkp_ta_col = resolve_col(lookup_df, ["TA","Therapeutic Area","TA_NAME"])
if lkp_ta_col is None:
    raise ValueError(f"Could not find TA column in {LOOKUP_TABLE}")

# Deduplicate so mapping is deterministic (keep first non-null)
lookup_df = lookup_df[[lkp_campaign_col, lkp_brandfile_col, lkp_ta_col]].copy()
lookup_df = lookup_df.dropna(subset=[lkp_campaign_col]).drop_duplicates(subset=[lkp_campaign_col], keep="first")

BRAND_NAME_MAP = dict(zip(lookup_df[lkp_campaign_col].astype(str), lookup_df[lkp_brandfile_col].astype(str)))
TA_MAP = dict(zip(lookup_df[lkp_campaign_col].astype(str), lookup_df[lkp_ta_col].astype(str)))

folder_month_label = f"{calendar.month_name[MONTH]} {YEAR}"

def ensure_out_path(franchise, strat, ctype_val, month_name, channel_val,
                    ta_val=None, brand_file_name=None, start_day=None, end_day=None):
    """
    Build the output path. TA (from lookup) is included in filename.
    Franchise–TA de-duplication:
      - If identical (case-insensitive), keep one.
      - If they share the same 'family head' (before '_' or '-'), or one is a prefix of the other,
        prefer TA when it is the 'family' form (e.g., ONCOLOGY + ONC_Trodelvy → keep ONC_Trodelvy).
        Otherwise keep the shorter token.
      - Else keep both [Franchise, TA].
    If Campaign Type == UNBRANDED, include brand_file_name in filename.
    """
    channel_folder = (str(channel_val).strip() or "Unknown")
    safe_channel_folder = safe_token(channel_folder) or "Unknown"
    out_base = os.path.join(
        BASE_DIR,
        channel_folder if safe_channel_folder == channel_folder else safe_channel_folder,
        f"{folder_month_label} Data Feed Delivery"
    )
    os.makedirs(out_base, exist_ok=True)

    ctype_tc = title_case_str(ctype_val)  # "Branded" / "Unbranded" / etc.
    strat_up = (str(strat).strip().upper() or "NA")
    channel_token = safe_token(channel_val).upper() or "CHANNEL"
    month_token = safe_token(month_name)

    # Clean tokens
    franchise_token = safe_token(franchise)
    ta_token = safe_token(ta_val)

    def head(s: str) -> str:
        return re.split(r"[_\-]", s.upper())[0] if s else ""

    # Smart Franchise–TA merge
    if franchise_token and ta_token:
        fU, tU = franchise_token.upper(), ta_token.upper()
        fH, tH = head(franchise_token), head(ta_token)
        if fU == tU:
            lead_parts = [franchise_token]
        elif fH == tH or fH.startswith(tH) or tH.startswith(fH):
            if "_" in ta_token or "-" in ta_token:
                lead_parts = [ta_token]
            else:
                lead_parts = [franchise_token] if len(franchise_token) < len(ta_token) else [ta_token]
        else:
            lead_parts = [franchise_token, ta_token]
    elif franchise_token:
        lead_parts = [franchise_token]
    elif ta_token:
        lead_parts = [ta_token]
    else:
        lead_parts = ["NA"]

    parts = lead_parts + [
        safe_token(strat_up),
        channel_token,
        safe_token(ctype_tc if ctype_tc else "NA"),
    ]

    # Only for UNBRANDED include Brand File Name in filename.
    if ctype_tc == "Unbranded" and brand_file_name:
        parts.append(safe_token(brand_file_name))

    if start_day is None:
        parts.extend([month_token, str(YEAR)])
    else:
        parts.append(f"{month_token}_{start_day}-{end_day}_{YEAR}")

    filename = "_".join(parts) + ".csv"
    return os.path.join(out_base, filename)

# ----------------- PASS 1: counts per group (Brand File Name + TA from lookup) -----------------
group_counts = {}

for chunk in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
    # Detect key columns
    channel_col  = resolve_col(chunk, ["Channel","CHANNEL","channel"])  or "__CHANNEL__"
    if channel_col == "__CHANNEL__": chunk["__CHANNEL__"] = "Unknown"

    franchise_col = resolve_col(chunk, ["FRANCHISE","Franchise","FRANCHISE_NAME","Brand_Franchise","BRAND_FRANCHISE","BRAND"]) or "__FRANCHISE__"
    if franchise_col == "__FRANCHISE__": chunk["__FRANCHISE__"] = "NA"

    strat_col = resolve_col(chunk, ["HCP/DTP","HCP_DTP","Strategy"]) or "__STRATEGY__"
    if strat_col == "__STRATEGY__": chunk["__STRATEGY__"] = "NA"

    ctype_col = resolve_col(chunk, ["Campaign Type ","Campaign Type","CAMPAIGN_TYPE","BRAND_TYPE","Brand Type"]) or "__CTYPE__"
    if ctype_col == "__CTYPE__": chunk["__CTYPE__"] = "NA"

    date_src = resolve_col(chunk, ["DATE","Date","DAY","Day"])
    if date_src is None:
        raise ValueError("DATE column not found in chunk.")
    dts = pd.to_datetime(chunk[date_src], errors="coerce")
    if dts.isna().all():
        raise ValueError("DATE could not be parsed in chunk.")

    # Map via campaign to Brand File Name and TA (from lookup)
    campaign_col = resolve_col(chunk, ["CAMPAIGN_NAME","Campaign Name","Campaign","BRAND_INDICATION_CAMPAIGN","Brand Indication/Campaign"]) or "__CAMPAIGN__"
    if campaign_col == "__CAMPAIGN__":
        chunk["__CAMPAIGN__"] = "NA"
        campaign_col = "__CAMPAIGN__"

    chunk["__BRAND_FILE_NAME__"] = chunk[campaign_col].astype(str).map(BRAND_NAME_MAP).fillna("NA")
    chunk["__TA__"] = chunk[campaign_col].astype(str).map(TA_MAP).fillna("NA")
    chunk["__MONTH_NAME__"] = calendar.month_name[MONTH]

    # Group by TA (from lookup) and brand file name
    grp = chunk.groupby([franchise_col, strat_col, ctype_col, "__MONTH_NAME__", channel_col, "__BRAND_FILE_NAME__", "__TA__"], dropna=False).size()
    for key, cnt in grp.items():
        group_counts[key] = group_counts.get(key, 0) + int(cnt)

if not group_counts:
    print("No rows found for the chosen month + channels.")
    raise SystemExit(0)

group_files_needed = {k: num_files_from_count_alteryx(v) for k, v in group_counts.items()}
group_ranges = {k: month_day_ranges(YEAR, MONTH, group_files_needed[k]) for k in group_counts}

# ----------------- PASS 2: write incrementally -----------------
files_written = set()
rows_written_total = 0

for chunk in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
    channel_col  = resolve_col(chunk, ["Channel","CHANNEL","channel"])  or "__CHANNEL__"
    if channel_col == "__CHANNEL__": chunk["__CHANNEL__"] = "Unknown"

    franchise_col = resolve_col(chunk, ["FRANCHISE","Franchise","FRANCHISE_NAME","Brand_Franchise","BRAND_FRANCHISE","BRAND"]) or "__FRANCHISE__"
    if franchise_col == "__FRANCHISE__": chunk["__FRANCHISE__"] = "NA"

    strat_col = resolve_col(chunk, ["HCP/DTP","HCP_DTP","Strategy"]) or "__STRATEGY__"
    if strat_col == "__STRATEGY__": chunk["__STRATEGY__"] = "NA"

    ctype_col = resolve_col(chunk, ["Campaign Type ","Campaign Type","CAMPAIGN_TYPE","BRAND_TYPE","Brand Type"]) or "__CTYPE__"
    if ctype_col == "__CTYPE__": chunk["__CTYPE__"] = "NA"

    date_src = resolve_col(chunk, ["DATE","Date","DAY","Day"])
    dts = pd.to_datetime(chunk[date_src], errors="coerce")
    chunk["__MONTH_NAME__"] = calendar.month_name[MONTH]
    chunk["__DAY__"] = dts.dt.day

    # Map via campaign to Brand File Name and TA (from lookup)
    campaign_col = resolve_col(chunk, ["CAMPAIGN_NAME","Campaign Name","Campaign","BRAND_INDICATION_CAMPAIGN","Brand Indication/Campaign"]) or "__CAMPAIGN__"
    if campaign_col == "__CAMPAIGN__":
        chunk["__CAMPAIGN__"] = "NA"
        campaign_col = "__CAMPAIGN__"

    chunk["__BRAND_FILE_NAME__"] = chunk[campaign_col].astype(str).map(BRAND_NAME_MAP).fillna("NA")
    chunk["__TA__"] = chunk[campaign_col].astype(str).map(TA_MAP).fillna("NA")

    # Determine which schema to use for this channel
    # NOTE: keys are per-group, so we pick schema inside the group loop below.
    source_map_cache = {}

    # Group (includes TA from lookup)
    for keys, g in chunk.groupby([franchise_col, strat_col, ctype_col, "__MONTH_NAME__", channel_col, "__BRAND_FILE_NAME__", "__TA__"], dropna=False):
        franchise, strat, ctype_val, month_name, channel_val, brand_file_name, ta_val = keys

        # Choose schema based on channel
        ch_low = str(channel_val).strip().lower()
        schema = OUTPUT_SCHEMA_PUBLISHER_FIRST if ch_low in CHANNELS_PUBLISHER_FIRST else OUTPUT_SCHEMA_STANDARD

        # Build/Reuse source_map for this schema key (to avoid re-resolving every group)
        schema_key = tuple([c[0] for c in schema])
        if (ch_low, schema_key) not in source_map_cache:
            smap = {}
            for out_name, candidates, _dtype in schema:
                smap[out_name] = resolve_col(chunk, candidates)
            source_map_cache[(ch_low, schema_key)] = smap
        source_map = source_map_cache[(ch_low, schema_key)]

        n_files = group_files_needed.get(keys, 1)
        ranges = group_ranges.get(keys, [(1, calendar.monthrange(YEAR, MONTH)[1])])

        # Build standardized dataframe following the chosen schema
        out_frames = {}
        for out_name, _cands, dtype in schema:
            src = source_map[out_name]
            if src is not None and src in g.columns:
                col = g[src].copy()
            else:
                if dtype == "num":
                    col = pd.Series(0, index=g.index, dtype="Int64")
                elif dtype == "date":
                    col = pd.to_datetime(g[date_src], errors="coerce")
                else:
                    col = pd.Series("NA", index=g.index, dtype="object")
            out_frames[out_name] = col
        g_base = pd.DataFrame(out_frames)

        # Remove "Video Completes" for Display & Newsletter exports
        nonvideo = ch_low in NONVIDEO_CHANNELS
        if nonvideo:
            g_base = g_base.drop(columns=["Video Completes"], errors="ignore")

        # Sort by standardized "Date" column (not "DATE")
        if "Date" in g_base.columns:
            g_base.sort_values("Date", inplace=True)

        # Write file(s)
        if n_files == 1:
            out_path = ensure_out_path(
                franchise, strat, ctype_val, month_name, channel_val,
                ta_val=ta_val,
                brand_file_name=brand_file_name if title_case_str(ctype_val) == "Unbranded" else None
            )
            header = not os.path.exists(out_path)
            g_base.to_csv(out_path, index=False, mode="a", header=header)
            files_written.add(out_path)
            rows_written_total += len(g_base)
        else:
            dom = g["__DAY__"]
            for (start_day, end_day) in ranges:
                part = g_base.loc[(dom >= start_day) & (dom <= end_day)]
                if part.empty:
                    continue
                out_path = ensure_out_path(
                    franchise, strat, ctype_val, month_name, channel_val,
                    ta_val=ta_val,
                    brand_file_name=brand_file_name if title_case_str(ctype_val) == "Unbranded" else None,
                    start_day=start_day, end_day=end_day
                )
                header = not os.path.exists(out_path)
                part.to_csv(out_path, index=False, mode="a", header=header)
                files_written.add(out_path)
                rows_written_total += len(part)

print(f"Done. Files: {len(files_written)}  Total rows written: {rows_written_total}")


# SOCIAL

In [ ]:
BASE_DIR = r"C:\Users\tanzil.baraskar\OneDrive - Interpublic\Desktop\Data Feed File Downloads\Social"
YEAR = 2025
MONTH = 8

TABLE_A = "GILEAD.CUSTOM_OUTPUTS.DATAFEED_SOCIAL"
TABLE_B = "GILEAD.CUSTOM_OUTPUTS.GILEAD_LOOKUP_SOCIAL_BRAND_NAME"


# The exact headers (and order) you want in CSVs
DESIRED_HEADERS = [
    "Franchise",
    "Subsidiary",
    "Brand",
    "Brand Indication/Campaign",
    "Date",
    "Week Start",
    "Week Sort",
    "Campaign Type",
    "Campaign",
    "Campaign ID",
    "Site / Publisher",
    "Site / Publisher ID",
    "DMA",
    "DMA Code",
    "Ad Group",
    "Ad Group ID",
    "Target Audience",
    "Ad",
    "Ad ID",
    "Media Impressions",
    "Media Clicks",
    "Click through Rate",
    "Engagement",
    "Media Cost(USD)",
    "Video Views",
    "Video Completes",
    "GMID",
]

# ========= HELPERS =========
def safe_token(s) -> str:
    s = "" if s is None or (isinstance(s, float) and pd.isna(s)) else str(s)
    s = s.strip().replace(" ", "_")
    s = re.sub(r"[^\w\-.]", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s or "NA"

def title_case_str(s) -> str:
    s = "" if s is None or (isinstance(s, float) and pd.isna(s)) else str(s)
    return s.strip().lower().title()

def derive_strategy_from_fields(site_pub_val, campaign_val) -> str:
    s = "" if site_pub_val is None or (isinstance(site_pub_val, float) and pd.isna(site_pub_val)) else str(site_pub_val).lower()
    c = "" if campaign_val  is None or (isinstance(campaign_val,  float) and pd.isna(campaign_val))  else str(campaign_val).lower()
    if "dtc" in s or "_dtc_" in c: return "DTC"
    if "dtp" in s or "_dtp_" in c: return "DTP"
    if "hcp" in s or "_hcp_" in c: return "HCP"
    return "NA"

# ========= QUERY =========
# Harden JOIN (TRIM + UPPER), and coalesce TA to Brand when lookup missing
query = f"""
SELECT
  a."Franchise",
  a."Subsidiary",
  a."Brand",
  a."Brand Indication/Campaign",
  a."Date",
  a."Week Start",
  a."Week Sort",
  a."Campaign Type",
  a."Campaign",
  a."Campaign ID",
  a."Site/Publisher",
  a."Site/Publisher ID",
  a."DMA",
  a."DMA Code",
  a."Ad Group",
  a."Ad Group ID",
  a."Target Audience",
  a."Ad",
  a."Ad ID",
  a."Media Impressions",
  a."Media Clicks",
  a."Click Through Rate",
  a."Engagements",
  a."Media Cost(USD)",
  a."Video Views",
  a."Video Completes",
  COALESCE(NULLIF(b.TA, ''), a."Brand") AS TA,
  b.BRAND_FILE_NAME,
  b.STRATEGY AS LOOKUP_STRATEGY
FROM {TABLE_A} a
LEFT JOIN {TABLE_B} b
  ON UPPER(TRIM(a."Campaign")) = UPPER(TRIM(b.CAMPAIGN))
WHERE EXTRACT(YEAR FROM a."Date") = {YEAR}
  AND EXTRACT(MONTH FROM a."Date") = {MONTH}
"""

# ========= RUN =========
df = pd.read_sql(query, conn)

if df.empty:
    print(f"No rows for {MONTH}/{YEAR}.")
else:
    # --- Normalize column names to match DESIRED_HEADERS and create blanks as needed
    rename_map = {
        "Site/Publisher": "Site / Publisher",
        "Site/Publisher ID": "Site / Publisher ID",
        "Click Through Rate": "Click through Rate",  # align case/style
        "Engagements": "Engagement",
    }
    df = df.rename(columns=rename_map)

    # Ensure GMID exists as an empty column
    if "GMID" not in df.columns:
        df["GMID"] = ""

    # Ensure all desired headers exist (create empty if missing)
    for col in DESIRED_HEADERS:
        if col not in df.columns:
            df[col] = ""

    # --- Date parsing & labels
    dt = pd.to_datetime(df["Date"], errors="coerce")
    if dt.isna().all():
        raise ValueError("All 'Date' values are NaT after parsing; check source data.")
    df["__DATE_PARSED__"] = dt

    df["__MONTH_YEAR__"] = df["__DATE_PARSED__"].dt.strftime("%B_%Y")
    month_str = df["__DATE_PARSED__"].dt.strftime("%B").iloc[0]
    year_int = int(df["__DATE_PARSED__"].dt.year.iloc[0])
    folder_month_label = f"{month_str} {year_int}"
    df["__CHANNEL__"] = "Social"

    # --- Campaign type column resolution
    if "Campaign Type" in df.columns:
        COL_CTYPE = "Campaign Type"
    elif "Brand Type" in df.columns:
        COL_CTYPE = "Brand Type"
    else:
        COL_CTYPE = "__CTYPE__"
        df[COL_CTYPE] = "NA"

    # --- Strategy: prefer lookup, else derive
    if "LOOKUP_STRATEGY" not in df.columns:
        df["LOOKUP_STRATEGY"] = ""
    df["LOOKUP_STRATEGY"] = df["LOOKUP_STRATEGY"].fillna("")

    def _get_site_publisher(row):
        # Be resilient to either header variant (we normalized to "Site / Publisher")
        return row.get("Site / Publisher", "")

    df["__STRATEGY__"] = df.apply(
        lambda r: (r["LOOKUP_STRATEGY"].strip()
                   if isinstance(r["LOOKUP_STRATEGY"], str) and r["LOOKUP_STRATEGY"].strip()
                   else derive_strategy_from_fields(_get_site_publisher(r), r.get("Campaign", ""))),
        axis=1
    )

    # --- Ensure TA is never blank: TA -> Brand -> Franchise -> 'NA'
    df["TA"] = (
        df.get("TA", "")
          .astype(str).str.strip()
          .where(lambda s: s.ne(""), df.get("Brand", ""))
          .where(lambda s: s.astype(str).str.strip().ne(""), df.get("Franchise", "NA"))
    )

    # --- Brand file column presence flag
    brand_file_col = "BRAND_FILE_NAME" if "BRAND_FILE_NAME" in df.columns else None
    has_brand_file = brand_file_col is not None

    # --- Output folder (Social)
    out_base = os.path.join(BASE_DIR, f"{folder_month_label} Data Feed Delivery", "Social")
    os.makedirs(out_base, exist_ok=True)

    # --- Group & write
    group_cols = ["TA", "__STRATEGY__", "__CHANNEL__", COL_CTYPE, "__MONTH_YEAR__"]
    if has_brand_file:
        group_cols_with_brand = group_cols + [brand_file_col]
    else:
        df["__DUMMY__"] = ""
        group_cols_with_brand = group_cols + ["__DUMMY__"]

    files_written = 0
    rows_written = 0

    for keys, g in df.groupby(group_cols_with_brand, dropna=False):
        if has_brand_file:
            ta, strat, channel, ctype_val, month_year, brand_file = keys
        else:
            ta, strat, channel, ctype_val, month_year, _ = keys
            brand_file = ""

        # Filename rules:
        # - BRANDED:   TA_STRATEGY_CHANNEL_CTYPE_MONTHYEAR.csv
        # - UNBRANDED: TA_STRATEGY_CHANNEL_CTYPE_BRANDFILE_MONTHYEAR.csv
        # - Other:     TA_STRATEGY_CHANNEL_CTYPEorNA_MONTHYEAR.csv
        ctype_tc = title_case_str(ctype_val)
        ctype_upper = (ctype_tc or "").upper()
        if ctype_upper == "BRANDED":
            parts = [safe_token(ta), safe_token(strat), safe_token(channel), safe_token(ctype_tc), safe_token(month_year)]
        elif ctype_upper == "UNBRANDED":
            parts = [
                safe_token(ta),
                safe_token(strat),
                safe_token(channel),
                safe_token(ctype_tc),
                safe_token(brand_file),
                safe_token(month_year),
            ]
        else:
            parts = [safe_token(ta), safe_token(strat), safe_token(channel), safe_token(ctype_tc or "NA"), safe_token(month_year)]

        filename = "_".join([p for p in parts if p]) + ".csv"
        out_path = os.path.join(out_base, filename)

        # Column selection in desired order (now guaranteed to exist)
        present_headers = [c for c in DESIRED_HEADERS if c in g.columns and c in df.columns]
        if not present_headers:
            present_headers = list(g.columns)

        # Stable sort by parsed date (keep mixed-case headers)
        g_out = g[present_headers].copy()
        g_out = g_out.assign(__DATE_PARSED__=g["__DATE_PARSED__"]).sort_values(
            by="__DATE_PARSED__", na_position="last"
        ).drop(columns="__DATE_PARSED__")

        g_out.to_csv(out_path, index=False)
        files_written += 1
        rows_written += len(g_out)
        print(f"Wrote: {out_path}  (rows: {len(g_out)})")

    print(f"\nDone. Files: {files_written}  Total rows written: {rows_written}")

    # --- Optional: quick debug of why lookup missed (if any)
    # This lists up to 50 distinct campaigns where LOOKUP_STRATEGY was blank and we derived it
    derived_mask = df["LOOKUP_STRATEGY"].astype(str).str.strip().eq("")
    if derived_mask.any():
        print("\n🔎 Campaigns missing strategy in lookup (showing up to 50):")
        print(df.loc[derived_mask, "Campaign"].astype(str).str.strip().drop_duplicates().head(50).to_list())


# SUMMARY GENERATE

## UPDATE HE TARGET MONTH FOR THE MONTH YOU ARE RUNNING DATA 

In [ ]:
import calendar

TARGET_MONTH = 8          
TARGET_YEAR  = 2025       
OUTPUT_DIR   = r"C:\Users\tanzil.baraskar\OneDrive - Interpublic\Desktop\Data Feed File Downloads"
os.makedirs(OUTPUT_DIR, exist_ok=True)

MONTH_NAME   = calendar.month_name[TARGET_MONTH]
FILE_BASENAME = f"{MONTH_NAME} {TARGET_YEAR} Data Validation Summary.xlsx"
OUTPUT_PATH   = os.path.join(OUTPUT_DIR, FILE_BASENAME)

# ---- SQL templates (month is injected) ----
SQL_QUERIES = {
    "DCM": f"""
        SELECT 
            "Franchise" AS "Franchise",
            "Brand" AS "Brand",
            "Brand Indication/Campaign" AS "Brand Indication/Campaign",
            "HCP/DTP" AS "Strategy",
            "Campaign",
            "Channel" AS "Channel",
            CASE WHEN "Site/Publisher" = 'MATTERKIND' THEN 'Addressable' ELSE 'Direct' END AS "Addressable/Direct",
            "Site/Publisher" AS "Site/Publisher",
            MONTHNAME(date) || '_' || TO_CHAR(date, 'YYYY') AS month_year,
            SUM("Media Impressions") AS "Sum_Media Impressions",
            SUM("Media Clicks") AS "Sum_Media Clicks",
            SUM("Video Completes") AS "Sum_Video Completes",
            COUNT(*) AS "Count"
        FROM DATAFEED_DCM
        WHERE MONTH(date) = {TARGET_MONTH} AND YEAR(date) = {TARGET_YEAR}
        GROUP BY ALL
    """,

    "DCM_AUD": f"""
        SELECT 
            "Franchise" AS "Franchise",
            "Brand" AS "Brand",
            "Brand Indication/Campaign" AS "Brand Indication/Campaign",
            "HCP/DTP" AS "Strategy",
            "Campaign",
            "Channel" AS "Channel",
            CASE WHEN "Site/Publisher" = 'MATTERKIND' THEN 'Addressable' ELSE 'Direct' END AS "Addressable/Direct",
            "Site/Publisher" AS "Site/Publisher",
            MONTHNAME(date) || '_' || TO_CHAR(date, 'YYYY') AS month_year,
            SUM("Media Impressions") AS "Sum_Media Impressions",
            SUM("Media Clicks") AS "Sum_Media Clicks",
            SUM("Audio Completes") AS "Sum_Audio Completes",
            COUNT(*) AS "Count"
        FROM DATAFEED_DCM_AUDIO
        WHERE "Channel" = 'Audio' AND MONTH(date) = {TARGET_MONTH} AND YEAR(date) = {TARGET_YEAR}
        GROUP BY ALL
    """,

    "SearchKey": f"""
        SELECT 
            "Franchise",
            "Brand",
            "Brand Indication/Campaign",
            UPPER(SPLIT_PART("Campaign", '_', 5))  AS "Strategy",
            "Engine",
            MONTHNAME("Date") || '_' || TO_CHAR("Date", 'YYYY') AS month_year,
            SUM("Media Impressions") AS "Sum_Media Impressions",
            SUM("Media Clicks") AS "Sum_Media Clicks",
            SUM("Media Cost (USD)") AS "Sum_Media Cost (USD)",
            COUNT(*) AS "Count"
        FROM DATAFEED_SEARCH_KEY
        WHERE MONTH("Date") = {TARGET_MONTH} AND YEAR("Date") = {TARGET_YEAR}
        GROUP BY ALL
    """,

    "SearchGeo": f"""
        SELECT 
            "Franchise",
            "Brand",
            "Brand Indication/Campaign",
            UPPER(SPLIT_PART("Campaign", '_', 5))  AS "Strategy",
            "Engine",
            MONTHNAME("Date") || '_' || TO_CHAR("Date", 'YYYY') AS month_year,
            SUM("Media Impressions") AS "Sum_Media Impressions",
            SUM("Media Clicks") AS "Sum_Media Clicks",
            SUM("Media Cost (USD)") AS "Sum_Media Cost (USD)",
            COUNT(*) AS "Count"
        FROM DATAFEED_SEARCH_GEO
        WHERE MONTH("Date") = {TARGET_MONTH} AND YEAR("Date") = {TARGET_YEAR}
        GROUP BY ALL
    """,

    "Social": f"""
        SELECT 
            "Franchise" AS "Franchise",
            "Brand" AS "Brand",
            "Brand Indication/Campaign" AS "Brand Indication/Campaign",
            "HCP/DTP" AS "Strategy",
            "Campaign",
            "Site/Publisher" AS "Site/Platform",
            SUM("Media Cost(USD)") AS "Sum_Media Cost (USD)",
            SUM("Media Impressions") AS "Sum_Media Impressions",
            SUM("Media Clicks") AS "Sum_Media Clicks",
            SUM("Video Completes") AS "Sum_Video Completes",
            SUM("Video Views") AS "Sum_Video Views",
            SUM("Engagements") AS "Sum_Engagement",
            SUM("Click Through Rate") AS "Click Through Rate",
            COUNT(*) AS "Count"
        FROM DATAFEED_SOCIAL
        WHERE MONTH("Date") = {TARGET_MONTH} AND YEAR("Date") = {TARGET_YEAR}
        GROUP BY ALL
    """
}

# Explicit sheet order in the final workbook
SHEET_ORDER = ["DCM", "DCM_AUD", "SearchKey", "SearchGeo", "Social"]

def fetch_df(conn, sql: str) -> pd.DataFrame:
    cur = conn.cursor()
    try:
        cur.execute(sql)
        cols = [c[0] for c in cur.description]
        rows = cur.fetchall()
        return pd.DataFrame(rows, columns=cols)
    finally:
        cur.close()

def run_and_export_one_workbook(conn):
    # Create workbook with one sheet per query (in specified order)
    with pd.ExcelWriter(OUTPUT_PATH, engine="openpyxl") as xw:
        for name in SHEET_ORDER:
            sql = SQL_QUERIES[name]
            df = fetch_df(conn, sql)
            df.to_excel(xw, index=False, sheet_name=name)
            print(f"[{name}] rows: {len(df):,}")

    print(f"\n✅ Wrote: {OUTPUT_PATH}")

# ---- Run it ----
run_and_export_one_workbook(conn)


[DCM] rows: 249
[DCM_AUD] rows: 7
[SearchKey] rows: 43
[SearchGeo] rows: 43
[Social] rows: 110

✅ Wrote: C:\Users\tanzil.baraskar\OneDrive - Interpublic\Desktop\Data Feed File Downloads\May 2025 Data Validation Summary.xlsx


# FILE GENERATION FOR MONTHLY IW
## CHANGE MONTH LABEL

In [ ]:
MONTH_LABEL = "August 2025"  
BASE_OUT = Path(r"C:\Users\tanzil.baraskar\OneDrive - Interpublic\Desktop\Data Feed File Downloads")

# Existing monthly delivery folder name found under each source tree
SOURCE_DELIVERY_FOLDER = f"{MONTH_LABEL} Data Feed Delivery"

# New folder we’re creating
IW_DELIVERY_FOLDER = f"IW {MONTH_LABEL} Data Feed Delivery"

# Channel subfolders to create (matches your layout)
CHANNEL_FOLDERS = [
    "AUD", "CTV", "Custom", "Display", "FEP", "Newsletter", "Non-DMA AUD",
    "OLV", "SearchGeo", "SearchKey", "Social"
]

# Copy toggles
COPY_SOCIAL_AND_SEARCH = True
COPY_DCM = True
COPY_NON_DMA_AUD = True

# File types to copy (None = all)
EXT_WHITELIST = None  # e.g., {".csv", ".xlsx", ".txt"}

# Overwrite behavior
OVERWRITE = False  # False -> add " (1)" / " (2)" if name collides

# Source roots
SOURCE_ROOTS = {
    "Social": BASE_OUT / "Social",
    "SEARCH GEO": BASE_OUT / "SEARCH GEO",
    "SEARCH KEY": BASE_OUT / "SEARCH KEY",
    "DCM": BASE_OUT / "DCM",
    "NON-DMA AUD": BASE_OUT / "Non-DMA AUD", 
}

# Some sources have an extra nested folder inside the monthly delivery folder
NESTED_INSIDE_DELIVERY = {
    "Social": "Social",
    "SEARCH GEO": "SearchGeo",
    "SEARCH KEY": "SearchKey",
    # DCM and Non-DMA AUD usually flat; leave out unless yours has a nested folder
}


def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)
    return p


def should_copy(path: Path) -> bool:
    if not path.is_file():
        return False
    if path.name.startswith("~$"):  # skip temp Office files
        return False
    if EXT_WHITELIST is None:
        return True
    return path.suffix.lower() in EXT_WHITELIST


def unique_destination(dest: Path) -> Path:
    if OVERWRITE or not dest.exists():
        return dest
    stem, suffix = dest.stem, dest.suffix
    i = 1
    while True:
        candidate = dest.with_name(f"{stem} ({i}){suffix}")
        if not candidate.exists():
            return candidate
        i += 1


def find_delivery_dirs(root: Path, target_name: str):
    """Yield every directory named exactly target_name anywhere under root (case-insensitive)."""
    target_low = target_name.strip().lower()
    for dirpath, dirnames, _ in os.walk(root):
        if os.path.basename(dirpath).strip().lower() == target_low:
            yield Path(dirpath)


def copy_tree(src_dir: Path, dst_dir: Path) -> int:
    """Copy all files (respecting EXT_WHITELIST/OVERWRITE) from src_dir (recursive) to dst_dir."""
    if not src_dir.exists():
        return 0
    count = 0
    for item in src_dir.rglob("*"):
        if not should_copy(item):
            continue
        rel = item.relative_to(src_dir)
        out_path = dst_dir / rel
        ensure_dir(out_path.parent)
        out_path = unique_destination(out_path)
        shutil.copy2(item, out_path)
        count += 1
    return count


# -------- DCM channel detection (filename-based) --------
# Maps filename keywords to IW channel folder names.
# Example: any file whose name has whole-word "audio" routes to "AUD".
DCM_CHANNEL_REGEX = re.compile(
    r"\b(audio|display|ctv|custom|fep|newsletter|olv)\b",
    flags=re.IGNORECASE
)

DCM_KEYWORD_TO_FOLDER = {
    "audio": "AUD",          # your note: AUD is Audio
    "display": "Display",
    "ctv": "CTV",
    "custom": "Custom",
    "fep": "FEP",
    "newsletter": "Newsletter",
    "olv": "OLV",
}

def detect_dcm_channel(filename: str) -> str | None:
    """Return IW channel folder name for a DCM file, or None if no match."""
    name = Path(filename).stem  # ignore extension for matching
    m = DCM_CHANNEL_REGEX.search(name.replace("_", " "))
    if not m:
        return None
    keyword = m.group(1).lower()
    return DCM_KEYWORD_TO_FOLDER.get(keyword)


# -------------------- Main --------------------
def main():
    iw_root = ensure_dir(BASE_OUT / IW_DELIVERY_FOLDER)
    for ch in CHANNEL_FOLDERS:
        ensure_dir(iw_root / ch)

    print(f"✅ Created: {iw_root}")
    print("✅ Subfolders:")
    for ch in CHANNEL_FOLDERS:
        print(f"  - {iw_root / ch}")

    # Copy Social / Search
    if COPY_SOCIAL_AND_SEARCH:
        for key in ("Social", "SEARCH GEO", "SEARCH KEY"):
            src_root = SOURCE_ROOTS[key]
            found_any = False
            total = 0
            for delivery_dir in find_delivery_dirs(src_root, SOURCE_DELIVERY_FOLDER):
                found_any = True
                nested = NESTED_INSIDE_DELIVERY.get(key)
                search_dir = delivery_dir / nested if nested else delivery_dir
                if nested and not search_dir.exists():
                    search_dir = delivery_dir  # fallback

                if key == "Social":
                    dest = iw_root / "Social"
                elif key == "SEARCH GEO":
                    dest = iw_root / "SearchGeo"
                else:
                    dest = iw_root / "SearchKey"

                copied = copy_tree(search_dir, dest)
                total += copied
                print(f"↪ Copied {copied} files from {search_dir} → {dest}")
            if not found_any:
                print(f"⚠️ No '{SOURCE_DELIVERY_FOLDER}' found under {src_root}")
            else:
                print(f"📦 {key}: total files copied = {total}")

    # Copy DCM with channel routing
    if COPY_DCM:
        src_root = SOURCE_ROOTS["DCM"]
        found_any = False
        total = 0
        routed = 0
        skipped = []
        for delivery_dir in find_delivery_dirs(src_root, SOURCE_DELIVERY_FOLDER):
            found_any = True
            search_dir = delivery_dir  # typically flat
            for item in search_dir.rglob("*"):
                if not should_copy(item):
                    continue
                channel_folder = detect_dcm_channel(item.name)
                if not channel_folder:
                    skipped.append(str(item))
                    continue
                dest_dir = iw_root / channel_folder
                ensure_dir(dest_dir)
                out_path = unique_destination(dest_dir / item.name)
                shutil.copy2(item, out_path)
                total += 1
                routed += 1
        if not found_any:
            print(f"⚠️ No '{SOURCE_DELIVERY_FOLDER}' found under {src_root}")
        print(f"📦 DCM: files copied = {total}, routed = {routed}")
        if skipped:
            print("⚠️ DCM files skipped (no channel keyword match):")
            for s in skipped:
                print(f"  - {s}")

    # Copy Non-DMA AUD
    if COPY_NON_DMA_AUD:
        src_root = SOURCE_ROOTS["NON-DMA AUD"]
        found_any = False
        total = 0
        for delivery_dir in find_delivery_dirs(src_root, SOURCE_DELIVERY_FOLDER):
            found_any = True
            search_dir = delivery_dir  # assume flat; customize if you actually nest
            dest = iw_root / "Non-DMA AUD"
            copied = copy_tree(search_dir, dest)
            total += copied
            print(f"↪ Copied {copied} files from {search_dir} → {dest}")
        if not found_any:
            print(f"⚠️ No '{SOURCE_DELIVERY_FOLDER}' found under {src_root}")
        else:
            print(f"📦 Non-DMA AUD: total files copied = {total}")

    print("\n✅ Done. All copies complete. (No files moved.)")


if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        sys.exit("\nInterrupted by user.")


# MOVE FILES FOR IPSOS

### UPDATE THE MONTH LABEL AND DELIVERY FOLDER NAME

In [ ]:
MONTH_LABEL = "August 2025"            
DELIVERY_FOLDER = "August 2025 Data Feed Delivery"  
BASE_OUT = Path(r"C:\Users\tanzil.baraskar\OneDrive - Interpublic\Desktop\Data Feed File Downloads")

SOURCE_ROOTS = [
    BASE_OUT / "Social",
    BASE_OUT / "DCM",
    BASE_OUT / "SEARCH GEO",
    BASE_OUT / "SEARCH KEY",
]

DEST_ROOT = BASE_OUT / f"IPSOS {MONTH_LABEL} Data Feeds"

BRAND_FOLDERS = ["Biktarvy", "Epclusa", "Descovy", "Trodelvy", "Yeztugo"]

OVERWRITE = False       # If True, overwrite existing files. If False, auto-rename with (1), (2)...
DRY_RUN = False         # If True, print what would happen without copying


ROUTING_RULES = [
    (("HIV_Treatment",),                     "Biktarvy"),
    (("ONC_Trodelvy", "ONCOLOGY_Trodelvy"),  "Trodelvy"),
    (("HCV",),                               "Epclusa"),
    (("HIV_PrEP_LEN",),                      "Yeztugo"),
    (("HIV_PrEP",),                          "Descovy"),
]


EXT_WHITELIST = None


NESTED_DELIVERY_SUBFOLDER = {
    "SEARCH KEY": "SearchKey",
    "SEARCH GEO": "SearchGeo",
    "SOCIAL": "Social",
}

def ensure_dirs():
    DEST_ROOT.mkdir(parents=True, exist_ok=True)
    for b in BRAND_FOLDERS:
        (DEST_ROOT / b).mkdir(parents=True, exist_ok=True)


def normalize(s: str) -> str:
    """Case-insensitive, strip leading/trailing spaces for safer matching."""
    return s.strip().lower()


def choose_brand(filename: str) -> str | None:
    """Return brand folder name based on filename prefix rules; None if no match."""
    name = Path(filename).name  # just the file name
    low = normalize(name)
    for prefixes, brand in ROUTING_RULES:
        for p in prefixes:
            if low.startswith(normalize(p)):
                return brand
    return None


def find_delivery_dirs(root: Path, target_dirname: str):
    """Yield every directory named exactly target_dirname anywhere under root."""
    target_low = normalize(target_dirname)
    for dirpath, dirnames, _filenames in os.walk(root):
        # Check if current dir's name matches DELIVERY_FOLDER (case-insensitive)
        if normalize(os.path.basename(dirpath)) == target_low:
            yield Path(dirpath)


def unique_destination(dest: Path) -> Path:
    """If OVERWRITE=False and file exists, append ' (n)' before suffix to avoid clobbering."""
    if OVERWRITE or not dest.exists():
        return dest
    stem, suffix = dest.stem, dest.suffix
    i = 1
    while True:
        candidate = dest.with_name(f"{stem} ({i}){suffix}")
        if not candidate.exists():
            return candidate
        i += 1


def should_copy_file(p: Path) -> bool:
    if not p.is_file():
        return False
    if p.name.startswith("~$"):  # skip temp Office files
        return False
    if EXT_WHITELIST is None:
        return True
    return p.suffix.lower() in EXT_WHITELIST


# ========================== Main ===================================
def main():
    ensure_dirs()

    summary = defaultdict(int)
    skipped_no_brand = []
    scanned_delivery_dirs = []

    for src_root in SOURCE_ROOTS:
        if not src_root.exists():
            print(f"⚠️ Source root not found: {src_root}")
            continue

        src_key = src_root.name.upper()
        expected_nested = NESTED_DELIVERY_SUBFOLDER.get(src_key, None)

        for delivery_dir in find_delivery_dirs(src_root, DELIVERY_FOLDER):
            scanned_delivery_dirs.append(delivery_dir)

            # If this source expects a nested subfolder inside the delivery folder, use it.
            search_dir = delivery_dir
            if expected_nested:
                candidate = delivery_dir / expected_nested
                if candidate.exists() and candidate.is_dir():
                    search_dir = candidate
                else:
                    print(f"⚠️ Expected nested folder missing for {src_root.name}: {candidate}")

            for item in search_dir.rglob("*"):
                if not should_copy_file(item):
                    continue

                brand = choose_brand(item.name)
                if brand is None:
                    skipped_no_brand.append(str(item))
                    continue

                dest_dir = DEST_ROOT / brand
                dest_path = dest_dir / item.name
                dest_path = unique_destination(dest_path)

                if DRY_RUN:
                    print(f"[DRY RUN] COPY: {item} -> {dest_path}")
                else:
                    dest_dir.mkdir(parents=True, exist_ok=True)
                    shutil.copy2(item, dest_path)

                summary[brand] += 1

    # --------- Reporting ----------
    print("\n========== SUMMARY ==========")
    print(f"Scanned delivery folders matching '{DELIVERY_FOLDER}':")
    if scanned_delivery_dirs:
        for d in scanned_delivery_dirs:
            print(f"  - {d}")
    else:
        print("  (none found)")

    print("\nCopied files by brand:")
    for b in BRAND_FOLDERS:
        print(f"  {b}: {summary[b]}")

    if skipped_no_brand:
        print("\nSkipped (no routing rule matched):")
        for s in skipped_no_brand:
            print(f"  - {s}")

    if DRY_RUN:
        print("\nNote: DRY_RUN=True, no files were actually copied.")

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        sys.exit("\nInterrupted by user.")
